## 概要
将盒子上表面的3D点云投影在桌面所在的空间平面

```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import copy
import numpy as np
import cv2
import open3d as o3d
# 自定义库
from kyle_robot_toolbox.open3d import *

pybullet build time: Jun  3 2022 02:05:55


## 载入点云

In [2]:
# 场景点云
scene_pcd = o3d.io.read_point_cloud(f"./data/example/box_pose/pcd_rect_roi.pcd")

# 载入盒子表面点云
# 去除离群点后的
box_panel_pcd = o3d.io.read_point_cloud("./data/example/box_pose/box_panel_filter.pcd")

# 赋值为灰色
box_panel_pcd.paint_uniform_color([0.5, 0.5, 0.5])

# 点云可视化
draw_geometry([box_panel_pcd], window_name="移除离群点后的上表面点云")

## 载入桌面拟合平面

In [3]:
# 载入工作台的平面数据
desktop_panel_model = np.loadtxt("data/example/box_pose/desktop_panel_model.txt", delimiter=",")

[A, B, C, D] = desktop_panel_model
print(f"拟合桌面平面的表达式: {A:.2f}x + {B:.2f}y + {C:.2f}z + {D:.2f} = 0")

拟合桌面平面的表达式: 0.05x + 0.01y + 1.00z + -0.26 = 0


## 点云投影到空间平面

In [4]:
# 投影到空间平面上的点
box_underside_pcd = pcd_project2panel(box_panel_pcd, A, B, C, D, x0=0, y0=0, z0=0)
box_underside_pcd.paint_uniform_color([0.0, 1.0, 0])

PointCloud with 4436 points.

In [5]:
# 点云可视化
draw_geometry([box_panel_pcd, box_underside_pcd],\
                                  window_name="盒子下表面点云")

In [ ]:
# 点云可视化
draw_geometry([scene_pcd, box_underside_pcd],\
                                  window_name="盒子下表面点云")

In [ ]:
# 保存点云
o3d.io.write_point_cloud("./data/example/box_pose/box_underside_pcd.pcd", box_underside_pcd)

## 将点云投影在盒子上表面所在的平面

根据案例`05.盒子上表面中心点与高度测量`得到盒子的高度为0.076m，这里不再赘述。 
盒子上表面与桌面是平行关系，因此可以根据桌面平面表达式得到盒子上表面平面表达式。

也可以将`box_height`设置为游标卡尺的实际测量值。

In [ ]:
# 盒子高度，单位m
box_height = 0.03
# 盒子上表面的表达式
box_top_panel_model = [A, B, C, D + box_height] 

In [ ]:
A2, B2, C2, D2 = box_top_panel_model
# 投影到空间平面上的点
box_top_pcd = pcd_project2panel(box_panel_pcd, A2, B2, C2, D2, x0=0, y0=0, z0=0)
# 绘制为蓝色
box_top_pcd.paint_uniform_color([0.0, 0.0, 1.0])

In [ ]:
# 点云可视化
o3d.visualization.draw_geometries([scene_pcd, box_underside_pcd, box_top_pcd],\
                                  window_name="盒子上表面点云")